In [7]:
import plotly as py
import cufflinks as cf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime as dt
warnings.filterwarnings("ignore")

plt.style.use('ggplot')
print(cf.__version__)

# %config InlineBackend.figure_format = 'retina'
cf.go_offline()

0.17.0


In [222]:
dt_index = pd.DataFrame(pd.date_range(start='20100101', end='20191231')).rename(columns = {0 : 'date'})

def processing_Exchange_Rate(data):
    print('start')
    # 20100101부터 정렬
    data = data[::-1]
    # format datetime으로 변경
    data['날짜'] = data['날짜'].apply(lambda x : parse(x[:4] + x[6:8] + x[10:12]))
    # 20100101-20191231 인덱스 설정
    data = pd.merge(dt_index, data, how = 'left', left_on='date', right_on='날짜').set_index('date')
    print('전처리 전 null 갯수 : {}'.format(data['종가'].isnull().sum()))
    # Null값일 때 다음날 데이터가 있을 경우 오픈 데이터로 Null값 채우기
    data['Fill_value'] = data.shift(-1)['오픈']
    data['종가'] = data[['종가', 'Fill_value']].apply(lambda x : x['Fill_value'] if x['종가'] is np.nan else x, axis = 1)
    print('전처리 후 null 갯수 : {}'.format(data['종가'].isnull().sum()))
    
#     # 나머지 null 값 앞 뒤 평균으로 채우기
#     fillnull = pd.DataFrame()
#     fillnull['main'] = data['종가']
#     fillnull['back'] = data['종가'].fillna(method = 'bfill', limit=1)
#     fillnull['front'] = data['종가'].fillna(method = 'ffill', limit=1)
    
#     def fill_data(data):
#         if data['main'].isnan
#     fillnull.apply(fill_data, axis = 1)
    
                      
    print('전처리 후 null 갯수 : {}'.format(data['종가'].isnull().sum()))
    
    print('end')
    return data

In [223]:
Exchange_Rate = processing_Exchange_Rate(pd.read_csv('USD_KRW.csv'))
Exchange_Rate.head(5)

start
전처리 전 null 갯수 : 1044
전처리 후 null 갯수 : 522
전처리 후 null 갯수 : 522
end


,날짜,종가,오픈,고가,저가,변동 %,Fill_value
date,,,,,,,
2010-01-01,2010-01-01,"1,166.08","1,166.08","1,166.08","1,166.08",0.00%,NaN
2010-01-02,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-03,NaT,"1,164.07",NaN,NaN,NaN,NaN,"1,164.07"
2010-01-04,2010-01-04,"1,150.00","1,164.07","1,164.07","1,149.35",-1.38%,"1,151.00"
2010-01-05,2010-01-05,"1,140.20","1,151.00","1,151.00","1,136.30",-0.85%,"1,140.20"


In [261]:
fillnull['main'].fillna(0)
fillnull.head()

,main,back,front
date,,,
2010-01-01,"1,166.08","1,166.08","1,166.08"
2010-01-02,NaN,"1,164.07","1,166.08"
2010-01-03,"1,164.07","1,164.07","1,164.07"
2010-01-04,"1,150.00","1,150.00","1,150.00"
2010-01-05,"1,140.20","1,140.20","1,140.20"


In [263]:
fillnull = pd.DataFrame()
fillnull['main'] = Exchange_Rate['종가']
fillnull['main'].fillna('0', inplace = True)
fillnull['back'] = Exchange_Rate['종가'].fillna(method = 'bfill', limit=1)
fillnull['front'] = Exchange_Rate['종가'].fillna(method = 'ffill', limit=1)

def fill_data(data_fill):
    if data_fill['main'] == '0':
        return (data_fill['back'] + data_fill['front'])/ 2
    else:
        return data_fill['main']

fillnull.apply(fill_data, axis = 1)

TypeError: ("unsupported operand type(s) for /: 'str' and 'int'", 'occurred at index 2010-01-02 00:00:00')